Modelo

In [ ]:
PARAMETERS_default = {
    "u_o": 0,
    "u_u": 1.58,
    "theta_v": 0.3,
    "theta_w": 0.015,
    "theta_v_minus": 0.015,
    "theta_o": 0.006,
    "tau_v1_minus": 60.0,
    "tau_v2_minus": 1150,
    "tau_v_plus": 1.4506,
    "tau_w1_minus": 70,
    "tau_w2_minus": 20,
    "k_w_minus": 65.0,
    "u_w_minus": 0.03,
    "tau_w_plus": 280.0,
    "tau_fi": 0.11,
    "tau_o1": 6,
    "tau_o2": 6.0,
    "tau_so1": 43,
    "tau_so2": 0.2,
    "k_so": 2,
    "u_so": 0.65,
    "tau_s1": 2.7342,
    "tau_s2": 3.0,
    "k_s": 2.0994,
    "u_s": 0.9087,
    "tau_si": 2.8723,
    "tau_w_inf": 0.07,
    "w_inf_star": 0.94
}

# Standard Heaviside function
def H(x):
  if (x > 0.0):
    return 1.0
  else:
    return 0.0

# Functions for infinity values
def v_inf(u, params):
  if (u < params["theta_v_minus"]):
    return 1.0
  else:
    return 0.0

def w_inf(u, params):
  return (1.0 - H(u - params["theta_o"])) * (1.0 - u/params["tau_w_inf"]) + H(u - params["theta_o"]) * params["w_inf_star"]

# Functions for time constants
def tau_v_minus(u, params):
  return (1.0 - H(u - params["theta_v_minus"])) * params["tau_v1_minus"] + H(u - params["theta_v_minus"]) * params["tau_v2_minus"]

def tau_w_minus(u, params):
  tau_w1_minus = params["tau_w1_minus"]
  tau_w2_minus = params["tau_w2_minus"]
  k_w_minus = params["k_w_minus"]
  u_w_minus = params["u_w_minus"]
  return tau_w1_minus + (tau_w2_minus - tau_w1_minus) * (1.0 + np.tanh(k_w_minus * (u - u_w_minus))) / 2

def tau_so(u, params):
  k_so = params["k_so"]
  u_so = params["u_so"]
  tau_so1 = params["tau_so1"]
  tau_so2 = params["tau_so2"]
  return tau_so1 + (tau_so2 - tau_so1) * (1.0 + np.tanh(k_so * (u - u_so))) / 2

def tau_s(u, params):
  k_s = params["k_s"]
  u_s = params["u_s"]
  tau_s1 = params["tau_s1"]
  tau_s2 = params["tau_s2"]
  theta_w = params["theta_w"]
  return (1.0 - H(u - theta_w)) * tau_s1 + H(u - theta_w) * tau_s2

def tau_o(u, params):
  tau_o1 = params["tau_o1"]
  tau_o2 = params["tau_o2"]
  theta_o = params["theta_o"]
  return (1.0 - H(u - theta_o)) * tau_o1 + H(u - theta_o) * tau_o2

# Functions for currents

# novo metodo
def J_fi(u, v, params):
  theta_v = params["theta_v"]
  u_u = params["u_u"]
  tau_fi = params["tau_fi"]
  return -v * H(u - theta_v) * (u - theta_v) * (u_u - u) / tau_fi

def J_so(u, params):
  u_o = params["u_o"]
  theta_w = params["theta_w"]
  return (u - u_o) * (1.0 - H(u - theta_w)) / tau_o(u, params) + H(u - theta_w) / tau_so(u, params)

def J_si(u, w, s, params):
  theta_w = params["theta_w"]
  tau_si = params["tau_si"]
  return -H(u - theta_w) * w * s / tau_si

# RUSH LARSEN IMPLEMENTATION:

# s:
def tau_s_rl(u, params):
  k_s = params["k_s"]
  u_s = params["u_s"]
  theta_w = params["theta_w"]
  tau_s1 = params["tau_s1"]
  tau_s2 = params["tau_s2"]
  return (1.0 - H(u - theta_w)) * tau_s1 + H(u - theta_w) * tau_s2

def s_inf_rl(u, params):
  theta_w = params["theta_w"]
  u_s = params["u_s"]
  k_s = params["k_s"]
  return (1.0 + np.tanh(k_s * (u - u_s))) / 2

# v:
def tau_v_rl(u, params):
  theta_v = params["theta_v"]
  theta_v_minus = params["theta_v_minus"]
  tau_v1_minus = params["tau_v1_minus"]
  tau_v2_minus = params["tau_v2_minus"]
  tau_v_plus = params["tau_v_plus"]
  return (tau_v_plus * tau_v_minus(u, params)) / (tau_v_plus - tau_v_plus * H(u - theta_v) + tau_v_minus(u, params) * H(u - theta_v))

def v_inf_rl(u, params):
  theta_v = params["theta_v"]
  theta_v_minus = params["theta_v_minus"]
  tau_v_plus = params["tau_v_plus"]
  return (tau_v_plus * v_inf(u, params) * (1 - H(u - theta_v))) / (tau_v_plus - tau_v_plus * H(u - theta_v) + tau_v_minus(u, params) * H(u - theta_v))

# w:
def tau_w_rl(u, params):
  theta_w = params["theta_w"]
  theta_v = params["theta_v"]
  tau_w1_minus = params["tau_w1_minus"]
  tau_w2_minus = params["tau_w2_minus"]
  k_w_minus = params["k_w_minus"]
  u_w_minus = params["u_w_minus"]
  tau_w_plus = params["tau_w_plus"]
  return (tau_w_plus * tau_w_minus(u, params)) / (tau_w_plus - tau_w_plus * H(u - theta_w) + tau_w_minus(u, params) * H(u - theta_w))

def w_inf_rl(u, params):
  theta_w = params["theta_w"]
  theta_v = params["theta_v"]
  tau_w_plus = params["tau_w_plus"]
  return (tau_w_plus * w_inf(u, params) * (1 - H(u - theta_w))) / (tau_w_plus - tau_w_plus * H(u - theta_w) + tau_w_minus(u, params) * H(u - theta_w))

def min_model (bcl_s1, BCL, params):
    S1_interval = bcl_s1 # S1 tem sempre BCL de BCL
    S2_interval = BCL
    BCL_fixed = BCL
    dt = 0.1
    num_S1 = 8
    T = (S1_interval * num_S1) + (BCL * 1) # vai sempre fazer num_S1 S1, depois 1 S2
    N = int(T/dt) + 1
    time= np.zeros(N)
    t=0
    t_ini = 0
    i=0
    voltage = np.zeros(N)
    duration = 1.

    u = 0.
    v = 1.
    w = 1.
    s = 0.

    BCL = S1_interval
    S2_not_done = True

    while(t<T):

        if t>=t_ini and t<=t_ini+duration and S2_not_done:
            Jstim = 1.
        else:
            Jstim = 0

        u = u + dt * (- (J_fi(u, v, params) + J_so(u, params) + J_si(u, w, s, params)) + Jstim)
        if (tau_v_rl(u, params) > 10**-10):
            v = v_inf_rl(u, params) + (v - v_inf_rl(u, params)) * np.exp(-dt / tau_v_rl(u, params))
        else:
            v = v + dt * ((1.0 - H(u - params["theta_v"])) * (v_inf(u, params) - v) / tau_v_minus(u, params) - H(u - params["theta_v"]) * v / params["tau_v_plus"])
        if (tau_w_rl(u, params) > 10**-10):
            w = w_inf_rl(u, params) + (w - w_inf_rl(u, params)) * np.exp(-dt / tau_w_rl(u, params))
        else:
            w = w + dt * ((1.0 - H(u - params["theta_w"])) * (w_inf(u, params) - w) / tau_w_minus(u, params) - H(u - params["theta_w"]) * w / params["tau_w_plus"])
        if (tau_s(u, params) > 10**-10):
            s = s_inf_rl(u, params) + (s - s_inf_rl(u, params)) * np.exp(-dt / tau_s_rl(u, params))
        else:
            s = s + dt * (((1.0 + np.tanh(params["k_s"] * (u - params["u_s"])))) / 2 - s) / tau_s(u, params)

        voltage[i] = u
        time[i] = t
        t = t+dt
        i+=1
        if t > S1_interval * (num_S1 - 1):
          BCL = S2_interval
          if t > S1_interval * num_S1 + 1:
            S2_not_done = False
        if t>t_ini+duration:
            t_ini += BCL
        # if BCL == S2_interval and BCL != S1_interval:
        #   print(BCL)
    # print(f"T = {S1_interval * num_S1} + {BCL * 1} = {S1_interval * num_S1 + BCL * 1} - t = {int(t)}")
    return [voltage, time]




Tentando otimizar o modelo 

Gerar dados

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

def stim(bcl_s1, t,BCL):    
    S1_interval = bcl_s1
    S2_interval = BCL
    num_S1 = 8
    duration=1.0
    if t < S1_interval * (num_S1 - 1):
        Jstim = 1.0 if (t % S1_interval) < duration else 0.0
    elif t <= (S1_interval * num_S1 + duration):
        Jstim = 1.0 if ((t - S1_interval * (num_S1-1)) % S2_interval) < duration else 0.0
    else:
        Jstim = 0.0
    return Jstim


def model(t, y, S1_interval, S2_interval, num_S1, duration, params):
    u, v, w, s = y

    # Compute Jstim based on time intervals
    if t < S1_interval * (num_S1 - 1):
        Jstim = 1.0 if (t % S1_interval) < duration else 0.0
    elif t <= (S1_interval * num_S1 + duration):
        Jstim = 1.0 if ((t - S1_interval * (num_S1-1)) % S2_interval) < duration else 0.0
    else:
        Jstim = 0.0

    # Pre-compute parameters
    tau_v = tau_v_rl(u, params)
    tau_w = tau_w_rl(u, params)
    tau_S = tau_s(u, params)

    # Compute derivatives
    du_dt = - (J_fi(u, v, params) + J_so(u, params) + J_si(u, w, s, params)) + Jstim
    dv_dt = ((v_inf_rl(u, params) - v) / tau_v) if tau_v > 1e-10 else ((1.0 - H(u - params["theta_v"])) * (v_inf(u, params) - v) / tau_v_minus(u, params) - H(u - params["theta_v"]) * v / params["tau_v_plus"])
    dw_dt = ((w_inf_rl(u, params) - w) / tau_w) if tau_w > 1e-10 else ((1.0 - H(u - params["theta_w"])) * (w_inf(u, params) - w) / tau_w_minus(u, params) - H(u - params["theta_w"]) * w / params["tau_w_plus"])
    ds_dt = ((s_inf_rl(u, params) - s) / tau_S) if tau_S > 1e-10 else (((1.0 + np.tanh(params["k_s"] * (u - params["u_s"]))) / 2 - s) / tau_S)

    return [du_dt, dv_dt, dw_dt, ds_dt]

def min_model_opt(bcl_s1, BCL, params):
    S1_interval = bcl_s1
    S2_interval = BCL
    num_S1 = 8
    duration = 1.0
    T = (S1_interval * num_S1) + (BCL * 1)
    
    time_span = (0, T)
    time_points = [i for i in range(int(T))]
    
    # Initial conditions
    y0 = [0, 1, 1, 0]
    
    # Solve the differential equations
    sol = solve_ivp(
        fun=lambda t, y: model(t, y, S1_interval, S2_interval, num_S1, duration, params),
        t_span=time_span,
        y0=y0,
        method='RK23',
        t_eval=time_points,
        max_step=2
    )
    
    return sol.y[0], sol.t


import time

bcl=380
# Time the Euler method
start_time = time.time()
y_euler, t_euler = min_model(480, bcl, PARAMETERS_default)
elapsed_time_ivp = time.time() - start_time
print(f"solve old time: {elapsed_time_ivp:.4f} seconds")

# Time the solve_ivp method
start_time = time.time()
y_ivp, t_ivp = min_model_opt(480, bcl, PARAMETERS_default)
elapsed_time_euler = time.time() - start_time
print(f"new time: {elapsed_time_euler:.4f} seconds")

# Plotting both results
plt.figure(figsize=(12, 6))
plt.plot(t_euler, y_euler, label='Euler Method', color='red', linestyle='--')
plt.plot(t_ivp, y_ivp, label='solve_ivp', color='blue')

plt.xlabel('Time')
plt.ylabel('Voltage')
plt.title('Voltage Over Time: solve_ivp vs. Euler Method')
plt.grid(True)
plt.legend()
plt.show()
ST=[stim(480,T,380)for T in t_ivp]



plt.plot(t_ivp,ST)
plt.grid()
plt.show()

c=0
for i in ST:
    if i!=0:
        print(c)
    c+=1

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pylab
from math import *
import ap_features as apf
import random
import os
from SALib.sample import saltelli
from SALib.sample import sobol
from SALib.sample import latin

import csv
import time
# Nome do arquivo CSV
from datetime import datetime
def run_simulation(params,plot=False):
        resultados = []
        i = 0
        num_S1 = 8
        duration = 1.0
        BCL=BCL_max_limit = 400
        BCL_min_limit = 300
        BCL_d=5
        number_it = int((BCL_max_limit - BCL_min_limit) /BCL_d ) 

        j = 0
        maxdvdt = np.zeros(number_it)
        beat1_apd = np.zeros(number_it)
        beat1_di = np.zeros(number_it)
        
        while(BCL > BCL_min_limit and j < number_it): # até BCL de BCL_min_limit
          voltagem, time = min_model_opt(BCL_max_limit, BCL, params)
          ST=[stim(BCL_max_limit, T,BCL)for T in time]
        
          trace = apf.Beats(y=voltagem, t=time,pacing=ST)
          beats = trace.beats
          tam = len(beats)

          if tam<9:
            #resultados.append(-inf)
            print("skipping")
              #plt.show()
            fig, ax = plt.subplots()
            plt.plot(time, voltagem,"--", linewidth=1 ,label='U x t')

            for beat in beats:
                  ax.plot(beat.t, beat.y,"-")
            plt.plot(time,ST)
            ax.set_title("BCL = " + str(BCL))
            plt.show()
                   
            return [0,0,0,0,0,0]
          
            continue         
          else :
            if(plot):
            #plt.plot(ST)
            #plt.show()
              fig, ax = plt.subplots()
              plt.plot(time, voltagem,"--", linewidth=1 ,label='U x t')

              for beat in beats:
                  ax.plot(beat.t, beat.y)
              ax.set_title("BCL = " + str(BCL))
              plt.plot(time,ST)

              plt.show()
          
            beat1 = beats[-1]
            beat1_ant = beats[-2]
            # else:
                # beat1 = beats[-2]
                # beat1_ant = beats[-3]
            beat1_apd[j] = beat1.apd(90)
            beat1_di[j] = BCL - beat1_ant.apd(90)
            dt = 1
            dV = (abs(np.diff(voltagem) / dt))
            instant_S2 = int((BCL_max_limit * (num_S1-1) + BCL)/ dt) # limitando o calculo de maxdVdt a partir da aplicacao de S2 + duration
            dV_S2 = dV[instant_S2:]
            maxdvdt[j]=(np.max(dV_S2))
            BCL-=BCL_d
            j += 1

            # Calcular derivada de APD e CV
        #beat1_apd = beat1_apd[:-1]
        #beat1_di = beat1_di[:-1]
            # derivada = np.diff(beat1_apd) / np.diff(BCL_array)

        derivada = np.diff(beat1_apd) / np.diff(beat1_di)
        resultados.append(np.min(beat1_apd))
        resultados.append(np.max(beat1_apd))       
        resultados.append(np.max(maxdvdt))
        resultados.append(np.min(maxdvdt))
        resultados.append(np.max(derivada))
        derivada2 = np.diff(maxdvdt) / np.diff(beat1_di)
        resultados.append(np.max(derivada2))

        # resultados.append(np.min(maxdvdt))
        # resultados.append(np.mean(derivada))
        # resultados.append(np.min(derivada))
        
 

        if (len(beat1_di) != len(maxdvdt)):
              maxdvdt = maxdvdt[:-1]
       
        if(plot):


          plt.plot(beat1_di, beat1_apd, linewidth=1, marker='x', label='APD x DI')
          plt.show()

          plt.plot(beat1_di, maxdvdt, color='red', label='maxdvdt Numérica')
          plt.legend()
          plt.show()

        print(resultados)
        return resultados

nomes_variaveis_amostragem = ['u_u', 'theta_v', 'tau_v_plus', 'tau_w_plus', 'tau_fi', 'tau_so1', 'tau_so2', 'tau_si']
header = nomes_variaveis_amostragem + ['minAPD', 'maxAPD', 'MAXmaxdVdt','MINmaxdVdt', 'maxDerivada', 'maxDerivada2']
filename = 'result_latin_hyper_cube.csv'
num_samples = 30
seed = 43
filename = f"{num_samples}_{seed}_{filename}"

def update_parameters(original_dict, subset_dict, keys_to_update):
    for key in keys_to_update:
        if key in subset_dict:
            original_dict[key] = subset_dict[key]
    return original_dict


##90 a 110%
bounds_amostragem=[PARAMETERS_default[key]*np.array([0.9,1.1]) for key in nomes_variaveis_amostragem ]

# Definir os limites do problema
problem_amostragem = {
    'num_vars': len(nomes_variaveis_amostragem),
    'names': nomes_variaveis_amostragem,
    'bounds': bounds_amostragem
}

np.random.seed(seed)   # Sampling using latin hypercube
# param_values = saltelli.sample(problem_amostragem, num_samples)
param_values = sobol.sample(problem_amostragem, num_samples)
np.random.seed(seed)   # Sampling using latin hypercube

total=0
certo=0
def convert_to_dict(params_list, names):
    return {name: value for name, value in zip(names, params_list)}

with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        new_row = header
        writer.writerow(new_row)
for params in param_values:
    start_time = time.time()
    results = run_simulation(update_parameters(PARAMETERS_default,convert_to_dict(params,nomes_variaveis_amostragem),nomes_variaveis_amostragem), plot=False) 
    elapsed_time_euler = time.time() - start_time   
    
    inputs = params

    print(params)
    if results[0] != -inf:
      with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        new_row = list(inputs) + list(results)
        writer.writerow(new_row)
    total+=1
    print(f"{total} de {len(param_values)} in {elapsed_time_euler}")


print(f"Dados salvos em {filename}")

In [ ]:
# times=np.linspace(0,4000,40000)
# ST=[stim(T,380)for T in times]

# ST2=[stim(T,380)for T in times]

# plt.plot(times,ST)
# plt.plot(times,ST2)


In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pylab
from math import *
import ap_features as apf
import random
import os
from SALib.sample import saltelli
from SALib.sample import sobol
from SALib.sample import latin

import csv
import time
# Nome do arquivo CSV
from datetime import datetime
def run_simulation(params,plot=False):
        resultados = []
        i = 0
        num_S1 = 8
        duration = 1.0
        BCL=BCL_max_limit = 400
        BCL_min_limit = 300
        BCL_d=5
        number_it = int((BCL_max_limit - BCL_min_limit) /BCL_d ) 

        j = 0
        maxdvdt = np.zeros(number_it)
        beat1_apd = np.zeros(number_it)
        beat1_di = np.zeros(number_it)
      

        while(BCL > BCL_min_limit and j < number_it): # até BCL de BCL_min_limit
          voltagem, time = min_model_opt(BCL_max_limit, BCL, params)
          ST=[stim(BCL_max_limit, T,BCL)for T in time]
        
          trace = apf.Beats(y=voltagem, t=time,pacing=ST)
          beats = trace.beats
          tam = len(beats)

          if tam<9:
            #resultados.append(-inf)
            print("skipping")
              #plt.show()
            fig, ax = plt.subplots()
            plt.plot(time, voltagem,"--", linewidth=1 ,label='U x t')

            for beat in beats:
                  ax.plot(beat.t, beat.y,"-")
            plt.plot(time,ST)
            ax.set_title("BCL = " + str(BCL))
            plt.show()
                   
            return [0,0,0,0,0,0]
          
            continue         
          else :
            if(plot):
            #plt.plot(ST)
            #plt.show()
              fig, ax = plt.subplots()
              plt.plot(time, voltagem,"--", linewidth=1 ,label='U x t')

              for beat in beats:
                  ax.plot(beat.t, beat.y)
              ax.set_title("BCL = " + str(BCL))
              plt.plot(time,ST)

              plt.show()
          
            beat1 = beats[-1]
            beat1_ant = beats[-2]
            # else:
                # beat1 = beats[-2]
                # beat1_ant = beats[-3]
            beat1_apd[j] = beat1.apd(90)
            beat1_di[j] = BCL - beat1_ant.apd(90)
            dt = 1
            dV = (abs(np.diff(voltagem) / dt))
            instant_S2 = int((BCL_max_limit * (num_S1-1) + BCL)/ dt) # limitando o calculo de maxdVdt a partir da aplicacao de S2 + duration
            dV_S2 = dV[instant_S2:]
            maxdvdt[j]=(np.max(dV_S2))
            BCL-=BCL_d
            j += 1

            # Calcular derivada de APD e CV
        #beat1_apd = beat1_apd[:-1]
        #beat1_di = beat1_di[:-1]
            # derivada = np.diff(beat1_apd) / np.diff(BCL_array)

        derivada = np.diff(beat1_apd) / np.diff(beat1_di)
        # resultados.append(np.min(beat1_apd))
        # resultados.append(np.max(beat1_apd))       
        # resultados.append(np.max(maxdvdt))
        # resultados.append(np.min(maxdvdt))
        # resultados.append(np.max(derivada))
        # derivada2 = np.diff(maxdvdt) / np.diff(beat1_di)
        # resultados.append(np.max(derivada2))

        # resultados.append(np.min(maxdvdt))
        # resultados.append(np.mean(derivada))
        # resultados.append(np.min(derivada))
        
 

        if (len(beat1_di) != len(maxdvdt)):
              maxdvdt = maxdvdt[:-1]

        # Criar a primeira figura e eixo
        # fig1 = plt.figure()
        # ax1 = fig1.add_subplot(111)
        ax1.plot(time, voltagem, linewidth=1, color='black',label='U x t')
        # ax1.set_xlabel("Tempo")
        # ax1.set_ylabel("Voltagem")
        # ax1.set_title("Voltagem x Tempo")
        # ax1.legend()
        # plt.savefig(f'{output_dir}/U_x_t.png')  # Salvar a imagem
        # plt.close(fig1)  # Fecha a figura após salvar

        # Criar a segunda figura e eixo
        # fig2 = plt.figure()
        # ax2 = fig2.add_subplot(111)
        ax2.plot(beat1_di, beat1_apd, linewidth=1, marker='x', color='black',label='APD x DI')
        # ax2.set_xlabel("Intervalo de Estímulo")
        # ax2.set_ylabel("Período de Ação")
        # ax2.set_title("APD x DI")
        # ax2.legend()
        # plt.savefig(f'{output_dir}/APD_x_DI.png')  # Salvar a imagem
        # plt.close(fig2)  # Fecha a figura após salvar

        # Criar a terceira figura e eixo
        # fig3 = plt.figure()
        # ax3 = fig3.add_subplot(111)
        ax3.plot(beat1_di, maxdvdt, color='black',label='maxdvdt')
        # ax3.set_xlabel("Intervalo de Estímulo")
        # ax3.set_ylabel("dV/dt Máx")
        # ax3.set_title("maxdvdt x Intervalo de Estímulo")
        # ax3.legend()
        # plt.savefig(f'{output_dir}/maxdvdt_num.png')  # Salvar a imagem
        # plt.close(fig3)  # Fecha a figura após salvar

        if(plot):

          plt.plot(time, voltagem, linewidth=1 ,label='U x t', color='black')
          plt.show()

          plt.plot(beat1_di, beat1_apd, linewidth=1, marker='x', label='APD x DI', color='black')
          plt.show()

          plt.plot(beat1_di, maxdvdt, label='maxdvdt Numérica', color='black')
          plt.legend()
          plt.show()

        # print(resultados)
        return [0,0,0,0,0,0]


fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15))

# Diretório para salvar as imagens
output_dir = 'imagens_simulacao'  # Nome do diretório de saída
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

nomes_variaveis_amostragem = ['u_u', 'theta_v', 'tau_v_plus', 'tau_w_plus', 'tau_fi', 'tau_so1', 'tau_so2', 'tau_si']
header = nomes_variaveis_amostragem + ['minAPD', 'maxAPD', 'MAXmaxdVdt','MINmaxdVdt', 'maxDerivada', 'maxDerivada2']
filename = 'result_latin_hyper_cube.csv'
num_samples = 30
seed = 50
filename = f"{num_samples}_{seed}_{filename}"

def update_parameters(original_dict, subset_dict, keys_to_update):
    for key in keys_to_update:
        if key in subset_dict:
            original_dict[key] = subset_dict[key]
    return original_dict


##90 a 110%
bounds_amostragem=[PARAMETERS_default[key]*np.array([0.9,1.1]) for key in nomes_variaveis_amostragem ]

# Definir os limites do problema
problem_amostragem = {
    'num_vars': len(nomes_variaveis_amostragem),
    'names': nomes_variaveis_amostragem,
    'bounds': bounds_amostragem
}

np.random.seed(seed)   # Sampling using latin hypercube
# param_values = saltelli.sample(problem_amostragem, num_samples)
param_values = sobol.sample(problem_amostragem, num_samples)
np.random.seed(seed)   # Sampling using latin hypercube

total=0
certo=0
def convert_to_dict(params_list, names):
    return {name: value for name, value in zip(names, params_list)}

with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        new_row = header
        writer.writerow(new_row)
for params in param_values:
    start_time = time.time()
    results = run_simulation(update_parameters(PARAMETERS_default,convert_to_dict(params,nomes_variaveis_amostragem),nomes_variaveis_amostragem), plot=False) 
    elapsed_time_euler = time.time() - start_time   

    inputs = params

    print(params)
    if results[0] != -inf:
      with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        new_row = list(inputs) + list(results)
        writer.writerow(new_row)
    total+=1
    print(f"{total} de {len(param_values)} in {elapsed_time_euler}")

plt.savefig(f'{output_dir}/combined_plots_{num_samples}_{seed}.png')  # Salvar a imagem
plt.close(fig)  # Fecha a figura após salvar
print(f"Dados salvos em {filename}")